# Solving Single Decisions

## The "Party Problem" example

JMA 11 Jan 2024

In [ ]:
# Imports from the python standard library
import math, re, os, sys
from pathlib import Path
import itertools            # to flatten lists

# Import array and dataframe packages
import numpy as np
# import numpy.linalg as la
import pandas as pd

import networkx as nx

# for extract_net
# from ID_operations import * 
from potential_operations import *
import BN

# Import the bokeh python wrappers for javascript plots
#  - a preferred visualization tool
# from bokeh.plotting import figure, show
# from bokeh.models import ColumnDataSource, VBar, Span
# from bokeh.io import output_notebook
# output_notebook()

NETWORK_FILE = 'PartyProblem_asym.xdsl' # 'PartyProblem_asym.xdsl'  # 

In [ ]:
# BN structure is contained under the node branch
parsed = BN.extract_net(NETWORK_FILE)
nodes, extensions = parsed
# tags tell the node type. 
[( k.get('id'), k.tag) for k in nodes]

## Bayes networks object

### Include state and variable names to tensor dimensions

### Add graph structure

It contains 

- the parse of the network as a dictionary with node names as keys
- The graph object showing network structure
- Node Potential objects for computation. 


In [ ]:
# CPT contents are stored in row major order (first row, second row, ...)
# Parents are the first matrix dimension -- matrix is Row Markov
pp_net = BN.reap(parsed)
pp_net.pr_nodes()


In [ ]:
pp_net.pr_influences()



### Extract matrices as tensors.  _List all tensors_

In [ ]:
pp_net.pr_named_tensors()

In [ ]:
# TODO move to BN class

pp_net.pr_network()

In [ ]:
## pretty print one of the variables
pp_net.pr_one_dim_table( 'Weather', tablefmt= '.4f', headers= ['State', 'Value'])



### Utilities

In [ ]:
# Utility matrix, as a Potential
outcome_potential = pp_net.get_potential('Preferences')
outcome_potential.pr_potential()

In [ ]:
utility_p = named_tensor_apply(outcome_potential, delta_utility, exponand = 0.5, normalize = 50)
utility_p.pr_potential()

In [ ]:
# Check that the inverse function returns the outcome values
named_tensor_apply(utility_p, delta_inverse_utility, exponand = 0.5, normalize = 50)

## Solving the party problem

_Using just potential operations (not Node removal)

To determine the optimal policy --

* join Adjustor and Detector CPTs, marginalize out Adjustor
* join Detector and Weather CPTs, marginalize out Detector 
* join Weather with Utility (Decision has unit values for all options)
* marginalize out unobserved Weather 
* Maximize over options
* (marginalize out Utility to get decision lottery)

### First solution - only prior, no observation 

In [ ]:
# Remove Adjustor
detector_p = pp_net.get_potential('Detector')
adjustor_p = pp_net.get_potential('Adjustor')
detector_marginal = absorb_parent(adjustor_p, detector_p)
detector_marginal.pr_potential()

In [ ]:
# Remove Detector
# Note this just returns the Weather prior, as it should. 
weather_p = pp_net.get_potential('Weather')
wd_joint = join(weather_p, detector_marginal)  #TODO wrap these in a reverse arc & remove function
weather_marginal = marginalize(wd_joint, 'Detector')
weather_marginal.pr_potential()

### Compute the expected outcome value

The expected value __before__ applying the utility function

Note that the optimal choice is implicit in the EV potential. The party location table is not used here. 

In [ ]:
pp_net.get_potential("Party_location")

In [ ]:
outcome_potential.pr_potential()

In [ ]:

expected_outcome = marginalize_utility(outcome_potential, weather_marginal)
alternatives = pp_net.get_node('Party_location').get_states()
print('Alternatives:', alternatives)
expected_outcome.pr_potential()

In [ ]:
v, ix = expected_outcome.cpt.max(dim=0)
alternatives[ix], v.item()

In [ ]:
# Compute optimal policy from expected outcome. 
# Find decision dimension (use 'd' as a type?)
# maximize over this dimension
# Replace decision policy with 0-1 values indicating optimal policy. 
torch.max(expected_outcome.cpt, dim=0)

In [ ]:
# We must drop the singleton dim first from the utility?
utility_p.pr_potential()

In [ ]:
# Utility expected value over decision alternatives
expected_utility = marginalize_utility(utility_p, weather_marginal)
print('Alternatives:',pp_net.get_node('Party_location').get_states())
expected_utility.pr_potential()



In [ ]:
v, ix = expected_utility.cpt.max(dim=0)
alternatives[ix], f'{v.item():.3f} '

### Party problem 2; when Weather is observed

To solve this --

* Add an informational "cause" to the decision node by
* Using the Weather marginal as a conditioning for Party location

_alternately add the conditioning arc in the xdsl file instead of programmatically modifying it._

In [ ]:
# P( Weather | Detector) - column markov
# See p 270 Figure 13.6
# Modify the decision node, and add the weather dimension to its Potential. 
decn_node = pp_net.get_node('Party_location')
decn_p = decn_node.get_potential()
decn_dims = decn_p.get_named_dims()
# Prefix the weather 'm' dim OrderedDict has a function for this
decn_dims['Weather'] = 'm'
decn_dims.move_to_end('Weather', last=False)
# Prefix a dimension to the decision cpt table
# Use the global weather potential
conditioning_size = weather_p.get_dim_sizes()[-1]    # Marginal dim is last
extended_shape = list(decn_p.get_dim_sizes())
extended_shape.insert(0, conditioning_size)
cpt = torch.ones(extended_shape)
extended_decn_p = Potential(cpt, decn_dims)

_Note: Create the modified net and draw its graph_ 


In [ ]:
#  weather and utility
# join utility and extended decn? / maximize decn, join & marginalize weather? 
squeezed_utility

In [ ]:
# One approach is to add a unsqueeze dim to match Detector at the end of preference transpose
# BINGO
extended_preference = preference_transpose.p.unsqueeze(-1).unsqueeze(-1)
print(extended_preference.shape)
# Sum out the weather dimension
policy_values = (extended_preference * posterior.p).sum(2)
print('E[ V | Party_location, Detector] = ')
policy_values
# Next we need to weight the optimal in each column by the pre-posterior.

In [ ]:
extended_preference * posterior.p

In [ ]:
# TODO Need to format list entries before passing to tabulate. 
# TODO looks like the State labels are flipped. 
detector_states= pp_net.n_dict['Detector']['states'].copy()
detector_states.insert(0, 'State')
pr_one_dim_table(policy_values.squeeze(0), 
    'Party_location',
    pp_net.n_dict, 
    floatfmt= ".3f", 
    headers= detector_states)
        

In [ ]:
policy_values.squeeze(0).sum(-1)

In [ ]:
# TODO what is the last dim?  Need to remove it. 
fig, ax = plt.subplots(1,2, figsize = (6, 2.6))
policy_values_2d_a = pd.DataFrame(policy_values.squeeze(0)[:,:,1], columns = pp_net.n_dict['Detector']['states'], 
                                index = pp_net.n_dict['Party_location']['states'])
sn.heatmap(policy_values_2d_a, annot=True, xticklabels=True, yticklabels=True, ax=ax[0])
policy_values_2d_b = pd.DataFrame(policy_values.squeeze(0)[:,:,0], columns = pp_net.n_dict['Detector']['states'], 
                                index = pp_net.n_dict['Party_location']['states'])
sn.heatmap(policy_values_2d_b, annot=True, xticklabels=True, yticklabels=True, ax=ax[1])

In [ ]:
# Find the max value in each column. 
decn = policy_values.max(1)
decn.values, decn.indices

In [ ]:
# Value with information. 
# However utility should be applied after computing expected values to get certain equivalents
# sigh
# 0.7782 * 0.44 + 0.6557 * 0.56
decn.values @ get_potential('Weather', pp_net.n_dict).p